In [1]:
import os
os.chdir("../..")
import pandas as pd
import numpy as np
from scripts.PdfParse import *
import datetime
import math

## Data Wrangling
### VU's aviation data

In [2]:
aviation_path = os.getcwd() + "/data/tourism/aviation_seats_flights_pic.xlsx"
aviation = pd.read_excel(aviation_path)
aviation.head(5)

,Country,ISO,Region,Date,Aircraft_type,Seats_arrivals_domestic,Seats_arrivals_interregional,Seats_arrivals_intraregional,Seats_arrivals_intl,Seats_arrivals_total,Available_seat_kilometers,Number_of_flights_domestic,Number_of_flights_interregional,Number_of_flights_intraregional,Number_of_flights_intl,Number_of_flights_total
0,Fiji,FJ,East Asia & Pacific,2019-01-01,passenger,839,273,3480,3753,4592,1.430416e+07,8,1,10,11,19
1,Fiji,FJ,East Asia & Pacific,2019-01-02,passenger,974,313,3471,3784,4758,1.495610e+07,8,1,10,11,19
2,Fiji,FJ,East Asia & Pacific,2019-01-03,passenger,1190,443,3675,4118,5308,1.592143e+07,10,2,12,14,24
3,Fiji,FJ,East Asia & Pacific,2019-01-04,passenger,831,586,3159,3745,4576,1.457334e+07,7,2,12,14,21
4,Fiji,FJ,East Asia & Pacific,2019-01-05,passenger,744,273,4752,5025,5769,1.773449e+07,7,1,12,13,20


In [3]:
vu_avi = (aviation[(aviation.ISO == "VU") & (aviation.Aircraft_type == "passenger")]
          .reset_index()
          .drop("index", axis=1)
          [["Date", "Number_of_flights_intl", "Number_of_flights_total",
            "Seats_arrivals_intl", "Seats_arrivals_total"]])

dates = pd.DataFrame(pd.date_range(start="2019-01-01",
                                   end="2022-10-16"), columns=["Date"])

vu_avi = dates.merge(vu_avi, how="left", on="Date")
vu_avi["Date"] = pd.to_datetime(vu_avi["Date"])
vu_avi["Month"], vu_avi["Year"] = vu_avi["Date"].dt.month, vu_avi["Date"].dt.year
vu_avi.head(5)

,Date,Number_of_flights_intl,Number_of_flights_total,Seats_arrivals_intl,Seats_arrivals_total,Month,Year
0,2019-01-01,4.0,4.0,552.0,552.0,1,2019
1,2019-01-02,4.0,4.0,584.0,584.0,1,2019
2,2019-01-03,3.0,3.0,408.0,408.0,1,2019
3,2019-01-04,4.0,4.0,674.0,674.0,1,2019
4,2019-01-05,4.0,4.0,652.0,652.0,1,2019


In [4]:
vu_avi_19_22 = vu_avi.groupby(by=["Year", "Month"]).sum().reset_index()
vu_avi_19_22.head(5)

,Year,Month,Number_of_flights_intl,Number_of_flights_total,Seats_arrivals_intl,Seats_arrivals_total
0,2019,1,123.0,123.0,18568.0,18568.0
1,2019,2,94.0,97.0,13680.0,14190.0
2,2019,3,101.0,101.0,14774.0,14774.0
3,2019,4,112.0,112.0,17071.0,17071.0
4,2019,5,111.0,111.0,16764.0,16764.0


### VU's Parsed Files

In [5]:
def generate_time(df: pd.DataFrame,
                  month_var: str,
                  year_var: str):

    import datetime

    for idx in df.index:
        time = df[month_var][idx]

        if type(time) == str and len(time) != 0:
            try:
                month_num = datetime.datetime.strptime(time, "%b").month
                if month_num == 1:
                    year = df[year_var][idx]
                    break
            except:
                pass

    year_lst, length = df[year_var].to_list(), len(df[year_var])
    for i in range(idx, length):
        year_lst[i] = (i-idx) // 12 + year

    return year_lst, idx

In [6]:
parsed_folder = os.getcwd() + "/data/tourism/vanuatu/byorigin/"
filepaths = os.listdir(parsed_folder)
filepaths = [parsed_folder + file for file in filepaths]

In [7]:
vu_parsed = pd.DataFrame()

for file in filepaths:
    filename = file.split("/")[-1]
    print(f"{filename} has started.")
    df = pd.read_csv(file)
    df = df.replace(r"\[p\]", "", regex=True).drop("Unnamed: 0", axis=1)
    df = df.rename({"Quarter": "Month",
                    "Chine": "China",
                    "Japon": "Japan",
                    "Australie": "Australia",
                    "Caledonia": "New Caledonia",
                    "Zealand": "New Zealand",
                    "America": "North America",
                    "Stated": "Not Stated",
                    "Countries": "Other Countries"}, axis=1)
    if "Month" and "Year" in df.columns:
        df["Year"], idx = generate_time(df, "Month", "Year")
        df = (df.iloc[idx:, ]
              .reset_index()
              .drop("index", axis=1))
        
        vu_parsed = (pd.concat([vu_parsed, df], axis=0)
                       .reset_index()
                       .sort_values(by="Year", ascending=True)
                       .drop("index", axis=1))
    else:
        pass
        print(f"{filename} has no Year or Month Column.")
    print(f"{filename} has ended.")

IVA_3_March_2022.csv has started.
IVA_3_March_2022.csv has ended.
IVA_7_July_2022.csv has started.
IVA_7_July_2022.csv has ended.
IVA_4_April_2022.csv has started.
IVA_4_April_2022.csv has ended.
IVA_1_January_2022.csv has started.
IVA_1_January_2022.csv has ended.
IVA_12_December-English_2019.csv has started.
IVA_12_December-English_2019.csv has ended.
IVA_5_May_2022.csv has started.
IVA_5_May_2022.csv has ended.
IAS_12_December_2018.csv has started.
IAS_12_December_2018.csv has ended.
IAS_12_Dececember_2017.csv has started.
IAS_12_Dececember_2017.csv has ended.
IVA_12_Dec_2020.csv has started.
IVA_12_Dec_2020.csv has ended.
2013-TM-12-December_News.csv has started.
2013-TM-12-December_News.csv has ended.
Tou12_December_2006.csv has started.
Tou12_December_2006.csv has ended.
TM12_December_2008_News.csv has started.
TM12_December_2008_News.csv has ended.
TM12_December_2009_News.csv has started.
TM12_December_2009_News.csv has ended.
IVA_12_December_2021.csv has started.
IVA_12_Decembe

In [8]:
month_equv = dict()

for i in range(1, 13):
    month_abbre = datetime.date(1900, i, 1).strftime('%b')
    month_full = datetime.date(1900, i, 1).strftime('%B')
    month_equv.update({month_full: i, month_abbre: i})

print(month_equv)

{'January': 1, 'Jan': 1, 'February': 2, 'Feb': 2, 'March': 3, 'Mar': 3, 'April': 4, 'Apr': 4, 'May': 5, 'June': 6, 'Jun': 6, 'July': 7, 'Jul': 7, 'August': 8, 'Aug': 8, 'September': 9, 'Sep': 9, 'October': 10, 'Oct': 10, 'November': 11, 'Nov': 11, 'December': 12, 'Dec': 12}


In [9]:
vu_parsed = (vu_parsed
             .replace(r"\*", "", regex=True)
             .drop_duplicates()
             .reset_index()
             .drop("index", axis=1)
             .fillna(0))

vu_parsed["month"] = vu_parsed["Month"].map(month_equv)


# Check the failed mappings
for idx in vu_parsed.index:
    month = vu_parsed["month"][idx]
    if math.isnan(month) == False:
        pass
    else:
        print(idx, vu_parsed["Year"][idx], vu_parsed["Month"][idx])


vu_parsed["month"] = vu_parsed["month"]
vu_parsed = vu_parsed.iloc[:-1].drop("month", axis=1)
vu_parsed.to_csv(
    os.getcwd() + "/data/tourism/vu_monthly_visitor.csv", encoding="utf-8")

266 2022.0 Ap


In [10]:
vu_19_22 = (vu_parsed[vu_parsed.Year >= 2019]
            .dropna(axis=1)
            .drop_duplicates()
            .reset_index()
            .drop("index", axis=1))


months = list()
for month in vu_19_22["Month"]:
    try:
        month = datetime.datetime.strptime(month, "%b").month
        months.append(month)
    except:
        try:
            month = datetime.datetime.strptime(month, "%B").month
            months.append(month)
        except:
            months.append(month)

vu_19_22["Month"] = months

In [11]:
## Manually drop the last row because of the typo
vu_19_22 = vu_19_22[:-1].sort_values(by=["Year", "Month"], ascending=True)
vu_19_22.head(5)

,Year,Month,Australia,New Zealand,New Caledonia,Other PIC,Europe,North America,Japan,China,Other Countries,Total,Not Stated
7,2019.0,1,5251,767,1786,481,523.0,229,61,284.0,166,9548,0.0
5,2019.0,2,2740,503,923,526,456.0,152,57,326.0,152,5835,0.0
9,2019.0,3,3864,611,949,411,413.0,209,71,331.0,167,7026,0.0
4,2019.0,4,5195,1010,1164,507,731.0,180,102,294.0,262,9445,0.0
8,2019.0,5,3919,868,1139,552,806.0,294,60,425.0,296,8359,0.0


## Correlation

In [12]:
vu_merged = vu_avi_19_22.merge(vu_19_22, how="left", on=["Year", "Month"]).dropna()
vu_merged

,Year,Month,Number_of_flights_intl,Number_of_flights_total,Seats_arrivals_intl,Seats_arrivals_total,Australia,New Zealand,New Caledonia,Other PIC,Europe,North America,Japan,China,Other Countries,Total,Not Stated
0,2019,1,123.0,123.0,18568.0,18568.0,5251.0,767.0,1786.0,481.0,523.0,229.0,61.0,284.0,166.0,9548.0,0.0
1,2019,2,94.0,97.0,13680.0,14190.0,2740.0,503.0,923.0,526.0,456.0,152.0,57.0,326.0,152.0,5835.0,0.0
2,2019,3,101.0,101.0,14774.0,14774.0,3864.0,611.0,949.0,411.0,413.0,209.0,71.0,331.0,167.0,7026.0,0.0
3,2019,4,112.0,112.0,17071.0,17071.0,5195.0,1010.0,1164.0,507.0,731.0,180.0,102.0,294.0,262.0,9445.0,0.0
4,2019,5,111.0,111.0,16764.0,16764.0,3919.0,868.0,1139.0,552.0,806.0,294.0,60.0,425.0,296.0,8359.0,0.0
5,2019,6,107.0,107.0,16088.0,16088.0,5525.0,1581.0,1101.0,607.0,808.0,256.0,52.0,390.0,287.0,10607.0,0.0
6,2019,7,107.0,107.0,15992.0,15992.0,6533.0,2344.0,1019.0,578.0,939.0,295.0,97.0,477.0,330.0,12612.0,0.0
7,2019,8,105.0,105.0,15904.0,15904.0,5700.0,2048.0,1354.0,753.0,931.0,322.0,157.0,531.0,479.0,12275.0,0.0
8,2019,9,103.0,103.0,15406.0,15406.0,6936.0,1526.0,1042.0,397.0,692.0,251.0,158.0,428.0,270.0,11700.0,0.0
9,2019,10,104.0,105.0,15599.0,15939.0,5679.0,1487.0,1307.0,567.0,795.0,305.0,69.0,414.0,257.0,10880.0,0.0


In [13]:
from scipy.stats import pearsonr

corr_seats, _ = pearsonr(vu_merged["Seats_arrivals_total"], vu_merged["Total"])
corr_seat_flight, _ = pearsonr(vu_merged["Number_of_flights_total"], vu_merged["Total"])
print(f"Pearson Correlation between FlightRadar's Seats Arrival and VU's census data is{corr_seats: .4f}.\n",
f"Pearson Correlation between FlightRadar's # of Flights Arrival and VU's census data is{corr_seat_flight: .4f}.")

Pearson Correlation between FlightRadar's Seats Arrival and VU's census data is 0.7161.
 Pearson Correlation between FlightRadar's # of Flights Arrival and VU's census data is 0.5080.


In [14]:
cols = ["Seats_arrivals_intl", "Seats_arrivals_total", "Number_of_flights_total", "Number_of_flights_intl", "Total"]
before_covid = vu_merged[:14][cols]
before_covid.corr()

,Seats_arrivals_intl,Seats_arrivals_total,Number_of_flights_total,Number_of_flights_intl,Total
Seats_arrivals_intl,1.000000,0.991323,0.991547,0.995984,0.551239
Seats_arrivals_total,0.991323,1.000000,0.995869,0.984069,0.554978
Number_of_flights_total,0.991547,0.995869,1.000000,0.992216,0.511768
Number_of_flights_intl,0.995984,0.984069,0.992216,1.000000,0.507492
Total,0.551239,0.554978,0.511768,0.507492,1.000000


## Bokeh Visualization

In [22]:
dates = list()
for year, month in zip(vu_parsed["Year"], vu_parsed["Month"]):
    date = str(year).replace(".0", "") + "-" + str(month) + "-01"
    dates.append(date)

vu_parsed["Date"] = dates
vu_parsed["Date"] = pd.to_datetime(vu_parsed["Date"])

In [29]:
from bokeh.palettes import Category20
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Legend


output_file('vu_month_by_country.html')
p = figure(height=600, width=1000,
           title="Number of Passengers Per Month",
           x_axis_type="datetime",
           x_axis_label="Date",
           y_axis_label="Number of Passengers Per Month",
           tooltips=[("Date", "$Date"),
                     ("Country", "$name"),
                     ("Passenger per month", "@$name")])

countries = vu_parsed.columns[~vu_parsed.columns.isin(["Year", "Month", "Total", "Date"])].to_list()
p.add_layout(Legend(), 'right')
source = ColumnDataSource(vu_parsed)

for country, color in zip(countries, Category20[12]):
    # have to use different colnames for y-coords so tooltip can refer to @$name
    p.line('Date', country, source=source, name=country,
           legend_label=country, color=color)

p.legend.label_text_font_size = '9pt'
p.legend.click_policy = "mute"
p.legend.location = "top_left"

show(p)